In [1]:
# !pip install repeng accelerate datasets matplotlib seaborn

In [2]:
#!/usr/bin/env python
# RASPID – “fluff PID” v2 + full AIME eval
# -------------------------------------------------
# 1) build contrastive control vector
# 2) train + calibrate fluff‐prob classifier
# 3) define baseline & RASPID generators
# 4) run run_aime_benchmark() and report

import os, random, math, re, warnings
import numpy as np, pandas as pd, torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from repeng import ControlModel, ControlVector, DatasetEntry
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

# ─── CONFIG & REPRO ────────────────────────────────────────────────────────
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
MODEL_NAME   = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE        = torch.float32
EMB_LAYER    = 20
CTRL_VEC_PT  = "ctrl_vector_clean_minus_fluff.pt"
MAX_CV_EX    = 800
MAX_CLF_EX   = 200
CHUNK_SIZES  = [16,64]
MAX_CHUNKS   = 5000
BATCH_SIZE   = 16
MAX_TOKENS   = 4096

# ─── LOAD MODELS ───────────────────────────────────────────────────────────
tokenizer    = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model   = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=DTYPE,
    device_map="auto" if DEVICE=="cuda" else None
).eval()
control_model = ControlModel(base_model,[EMB_LAYER])  # relies on base‘s device_map

# ─── 1) CONTRASTIVE CONTROL VECTOR ─────────────────────────────────────────
def build_contrastive_vector(n_ex=MAX_CV_EX, layer=EMB_LAYER):
    ds = load_dataset("rb/aime_reasoning","default")["train"].select(range(n_ex))
    clean_txt = [r["refined_reasoning"] for r in ds]
    fluff_txt = [r["reasoning_content"]   for r in ds]
    def embed(batch):
        toks = tokenizer(batch, padding=True, truncation=True,
                         max_length=256, return_tensors="pt").to(DEVICE)
        with torch.inference_mode():
            h = base_model(**toks, output_hidden_states=True
                          ).hidden_states[layer][:,0]
        return h.cpu()
    v = embed(clean_txt).mean(0) - embed(fluff_txt).mean(0)
    v = v / (v.norm()+1e-12)
    return ControlVector("qwen2", {layer: v.numpy()})

if os.path.exists(CTRL_VEC_PT):
    ctrl_vec = torch.load(CTRL_VEC_PT, map_location="cpu", weights_only=False)
    print("✅ loaded ctrl‐vec")
else:
    print("⚙️  training ctrl‐vec …")
    ctrl_vec = build_contrastive_vector()
    torch.save(ctrl_vec, CTRL_VEC_PT)
    print("💾 saved to", CTRL_VEC_PT)

# ─── 2) BUILD & TRAIN CHUNK FLUFF‐PROB CLASSIFIER ─────────────────────────
def build_chunk_dataset(cs):
    ds = load_dataset("rb/aime_reasoning","default")["train"].select(range(MAX_CLF_EX))
    chunks, labels = [], []
    for r in ds:
        for txt,lbl in [(r["refined_reasoning"],0),(r["reasoning_content"],1)]:
            ids = tokenizer.encode(txt)
            for i in range(0, len(ids)-cs+1, cs):
                chunks.append(tokenizer.decode(ids[i:i+cs], skip_special_tokens=True))
                labels.append(lbl)
    # cap total
    if len(chunks) > MAX_CHUNKS:
        idx = np.random.choice(len(chunks), MAX_CHUNKS, replace=False)
        chunks = [chunks[i] for i in idx]
        labels = [labels[i] for i in idx]
    return chunks, np.array(labels)

def embed_texts(texts, max_len):
    feats=[]
    for i in range(0, len(texts), BATCH_SIZE):
        toks = tokenizer(texts[i:i+BATCH_SIZE], padding=True, truncation=True,
                         max_length=max_len, return_tensors="pt").to(DEVICE)
        with torch.inference_mode():
            h = base_model(**toks, output_hidden_states=True
                          ).hidden_states[EMB_LAYER].mean(1)
        feats.append(h.cpu().numpy())
    return np.vstack(feats)

# grid‐search chunk size & platt scale
CALIB = {}
best_cs, best_acc, clf_best = None,0.0,None
for cs in CHUNK_SIZES:
    X_txt, y = build_chunk_dataset(cs)
    X_emb   = embed_texts(X_txt, cs)
    Xtr, Xval, ytr, yval = train_test_split(X_emb,y,test_size=0.2,
                                             random_state=SEED,stratify=y)
    clf = SGDClassifier(loss="log_loss",max_iter=600,tol=1e-4,random_state=SEED)
    clf.fit(Xtr,ytr)
    acc = accuracy_score(yval, clf.predict(Xval))
    print(f"chunk={cs:2d}  val_acc={acc*100:5.1f}%")
    if acc > best_acc:
        best_cs, best_acc, clf_best = cs, acc, clf
        raw = clf.decision_function(Xval).reshape(-1,1)
        platt = LogisticRegression(solver="lbfgs").fit(raw, yval)
        CALIB["a"], CALIB["b"] = platt.coef_[0,0], platt.intercept_[0]
print(f"▶ Using chunk_size={best_cs}, platt a={CALIB['a']:.3f}, b={CALIB['b']:.3f}")

# ─── 3) DEFINE GENERATORS ──────────────────────────────────────────────────
@torch.inference_mode()
def generate_baseline(prompt, max_tokens=MAX_TOKENS):
    inp = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    out = base_model.generate(
        **inp,
        max_new_tokens=max_tokens,
        do_sample=True, temperature=0.3,
        top_p=0.9, repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    toks = out.shape[1] - inp.input_ids.shape[1]
    return tokenizer.decode(out[0], skip_special_tokens=True), toks

@torch.inference_mode()
def generate_raspid(prompt, max_tokens=MAX_TOKENS,
                    chunk_size=best_cs, target_p=0.05,
                    kp=0.25, ki=0.002, kd=0.0005, max_alpha=1.0,
                    base_temp=1.0, steer_temp=0.3,
                    init_free=40, steer_window=60, max_repeat=8,
                    stop_regex=r"\\boxed\{[^{}]{1,12}\}", layer=EMB_LAYER):
    stop_re = re.compile(stop_regex)
    ids, past = tokenizer(prompt, return_tensors="pt").to(DEVICE).input_ids[0], None
    out_ids = ids.clone()
    α=I=D=prev_err=0.0
    chunk_hidd= None; tok_in_chunk=0
    steering_on=False; steer_start=0
    rep_ctr=0; last_tok=None
    a,b = CALIB["a"], CALIB["b"]

    print("\n*** RASPID LOG START ***\nchunk | p_fluff |  err  |   α   | temp")
    while len(out_ids)-len(ids) < max_tokens:
        L = len(out_ids)-len(ids)
        if not steering_on and L>=init_free:
            steering_on, steer_start = True, L
        if steering_on and L-steer_start > steer_window:
            steering_on, α, I, D = False,0,0,0

        control_model.set_control(ctrl_vec, coeff=α if steering_on else 0.)
        out = control_model(input_ids=out_ids[-1:].unsqueeze(0),
                            past_key_values=past, use_cache=True,
                            output_hidden_states=True)
        past = out.past_key_values
        logits = out.logits[0,-1]
        h_last = out.hidden_states[layer][0,-1]

        tok = out_ids[-1].item()
        rep_ctr = rep_ctr+1 if tok==last_tok else 0
        last_tok = tok
        if rep_ctr>=max_repeat: break

        chunk_hidd = h_last if chunk_hidd is None else chunk_hidd + h_last
        tok_in_chunk += 1

        if tok_in_chunk >= chunk_size:
            feat   = (chunk_hidd/chunk_size).cpu().unsqueeze(0).numpy()
            raw    = clf_best.decision_function(feat)[0]
            p_fluff= 1/(1+math.exp(- (a*raw + b) ))
            err    = p_fluff - target_p

            I    += ki*err
            D     = 0.9*D + 0.1*kd*(err-prev_err)
            prev_err = err
            α     = float(np.clip(kp*err + I + D, 0.0, max_alpha))
            temp  = base_temp*(1-α/max_alpha) + steer_temp*(α/max_alpha)
            print(f"{chunk_size:5d} | {p_fluff:7.3f} | {err:+.3f} | "
                  f"{α:5.3f} | {temp:5.3f}")

            chunk_hidd, tok_in_chunk = None, 0

        temp  = base_temp*(1-α/max_alpha) + steer_temp*(α/max_alpha)
        probs = torch.softmax(logits/temp, dim=-1)
        nxt   = torch.multinomial(probs,1).item()
        out_ids = torch.cat([out_ids, torch.tensor([nxt],device=DEVICE)])
        if stop_re.search(tokenizer.decode([nxt])) or nxt==tokenizer.eos_token_id:
            break

    print("*** RASPID LOG END ***\n")
    return tokenizer.decode(out_ids, skip_special_tokens=True), len(out_ids)-len(ids)

# ─── 4) EVAL HARNESS ──────────────────────────────────────────────────────
def normalize(ans: str):
    return re.sub(r"[^\d]", "", ans.strip())

def extract_answer(txt: str):
    m = re.search(r"\\boxed\{(\d+)\}", txt)
    return m.group(1) if m else ""

def run_aime_benchmark(n=10):
    ds_all = load_dataset("rb/aime_reasoning","default")["train"]
    reserved = set(range(MAX_CV_EX)) | set(range(MAX_CLF_EX))
    eval_idx = [i for i in range(len(ds_all)) if i not in reserved]
    if n>len(eval_idx): n=len(eval_idx)
    subs = random.sample(eval_idx, n)
    records=[]
    for i in tqdm(subs, desc="AIME eval"):
        q  = ds_all[i]["question"]
        ref= normalize(str(ds_all[i]["reference_answer"]))
        prompt = f"{q}\n\nAnswer step by step and end with: Final answer: \\boxed{{numeric_value}}"
        # baseline
        bt, btoks = generate_baseline(prompt, max_tokens=MAX_TOKENS)
        # raspid
        rt, rtoks = generate_raspid (prompt, max_tokens=MAX_TOKENS)
        ba = normalize(extract_answer(bt))
        ra = normalize(extract_answer(rt))
        records.append({
            "question":q,
            "reference":ref,
            "baseline_ans":bt, "raspid_ans":rt,
            "baseline_ok":ba==ref, "raspid_ok":ra==ref,
            "baseline_toks":btoks, "raspid_toks":rtoks
        })
    df = pd.DataFrame(records)
    print(f"\nBaseline acc={df.baseline_ok.mean()*100:.1f}%  "
          f"RASPID acc={df.raspid_ok.mean()*100:.1f}%  "
          f"Avg saving={(1-df.raspid_toks/df.baseline_toks).mean()*100:.1f}%")
    return df

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


✅ loaded ctrl‐vec


Token indices sequence length is longer than the specified maximum sequence length for this model (19690 > 16384). Running this sequence through the model will result in indexing errors


chunk=16  val_acc= 93.6%
chunk=64  val_acc= 95.6%
▶ Using chunk_size=64, platt a=0.003, b=4.373


In [3]:
df = run_aime_benchmark(n=1)
df.to_csv("aime_comparison.csv", index=False)
print(df.head())

AIME eval:   0%|          | 0/1 [00:00<?, ?it/s]


*** RASPID LOG START ***
chunk | p_fluff |  err  |   α   | temp
   64 |   0.782 | +0.732 | 0.184 | 0.871
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   0.999 | +0.949 | 0.239 | 0.833
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 | 0.832
   64 |   1.000 | +0.950 | 0.239 

AIME eval: 100%|██████████| 1/1 [02:17<00:00, 137.56s/it]

   64 |   1.000 | +0.950 | 0.239 | 0.832
*** RASPID LOG END ***


Baseline acc=100.0%  RASPID acc=100.0%  Avg saving=-109.6%
                                            question reference  \
0  The sum of the squares of the roots of the equ...             

                                        baseline_ans  \
0  The sum of the squares of the roots of the equ...   

                                          raspid_ans  baseline_ok  raspid_ok  \
0  The sum of the squares of the roots of the equ...         True       True   

   baseline_toks  raspid_toks  
0           1954         4096  


In [ ]:
ds = load_dataset("rb/aime_reasoning","default")["train"]

In [11]:
print(df['raspid_ans'].item())

The sum of the squares of the roots of the equation $x^2+2hx=3$ is $10$. The absolute value of $h$ is equal to
$\textbf{(A) }-1\qquad \textbf{(B) }\textstyle\frac{1}{2}\qquad \textbf{(C) }\textstyle\frac{3}{2}\qquad \textbf{(D) }2\qquad \textbf{(E) }\text{None of these}$

Answer step by step and end with: Final answer: \boxed{numeric_value} & & x \\, & & x \\, &, &, x \\, \end{array}}}$

I can summarize this in terms of a system of equations, such that:

$\frac{l}{x} + \frac{d}{x} + \frac{n}{x} = 1$

$l + d + n = N$

Wait, is that the correct way of putting it?

Alternatively, perhaps integrating the position x over the time period, as per cyclic motion.

But perhaps I can also consider Fourier transforms for the simultaneous analysis.

Let me try that.

First, define the sequence:

$a_0 = l$, $a_1 = d$, $a_2 = n$, $a_3 = x$

Then, in the frequency domain, the DFT is:

$X(k) = l + d + n + x$, for $k=0$

$X(k) = x \cdot (\omega^{-k} + \omega^{-2k} + \omega^{-3k} + 1)$, for $k=1,2,3$

Wh

In [5]:
ds[0]['reasoning_content']

"Okay, let's try to solve this problem step by step. The question is asking for the value of a nested absolute value expression when x is -2016. The expression is:\n\nBigg\\vert\\Big\\vert |x|-x\\Big\\vert-|x|\\Bigg\\vert - x\n\nAnd the options are given as (A) -2016, (B) 0, (C) 2016, (D) 4032, (E) 6048.\n\nFirst, let me parse the expression correctly. It's important to handle the absolute value signs properly, especially since they are nested. Let's break it down from the inside out.\n\nGiven that x = -2016, which is a negative number. Let's note that |x| is the absolute value of x, so |x| would be 2016 in this case.\n\nStarting with the innermost part: |x|. Since x is -2016, |x| = |-2016| = 2016.\n\nThen, the next part is |x| - x. Let's substitute the values here. We already know |x| is 2016, and x is -2016. So:\n\n|x| - x = 2016 - (-2016) = 2016 + 2016 = 4032.\n\nWait, that seems like a big number. Let me check that again. Absolute value of x is positive, so 2016. Then subtracting x

In [6]:
ds[0]['refined_reasoning']

'1. Given \\( x = -2016 \\), evaluate the expression \\( || |x| - x | - |x| | - x \\).\n2. Compute \\( |x| \\): \\( | -2016 | = 2016 \\).\n3. Evaluate \\( |x| - x \\): \\( 2016 - (-2016) = 2016 + 2016 = 4032 \\).\n4. Take the absolute value: \\( |4032| = 4032 \\).\n5. Subtract \\( |x| \\): \\( 4032 - 2016 = 2016 \\).\n6. Take the absolute value of the result: \\( |2016| = 2016 \\).\n7. Subtract \\( x \\): \\( 2016 - (-2016) = 2016 + 2016 = 4032 \\).\n8. Conclusively, the value of the expression is \\( 4032 \\), which corresponds to option (D).\n\nFinal answer: \\(\\boxed{D}\\)'